<a href="https://colab.research.google.com/github/jwood-iastate/CE-556-Fall-2021/blob/main/BL_Prob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from scipy.special import gamma
import math
import numpy as np

For a logit model, the probabilities that there are 1 or more crashes are computed as:

$P(y_i>0)=\frac{exp(\beta X)}{1+exp(\beta X)}$

Let's use a road segment for 2 consecutive weeks with $\beta X = -3.5$ for week 1 and $\beta X = -3.29$ for week 2. There are crashes on the road each week.

Also, remember Bayes theorem:

$P(A|B)=\frac{P(B|A)P(A)}{P(B)}$

Additionally, let's use:

$\mu=\beta X$

In [6]:
def logit_prob(mu):
  # returns the probability of 1 or more crashes
  P = np.exp(mu)/(1+np.exp(mu))
  return P

From the data, we know that the base probabilities, overall, are:

$$P(y>0)=0.02$$

$$P(y=0)=0.98$$

And:

$$y_1=1$$

$$y_2=2$$

Since we want to know the probability that a segment is non-randomly high (i.e., the segment has crash counts of 1 or more NOT due to randomness) we will define the outcomes:

$$A=High, but \ not \ randomly$$

$$B=High, or \ Y>0$$

We can assume that across all observations, $P(B)=P(High)$ is constant. For instance, we could define this as the "top 10%" (i.e., $P(A)=0.1$) or something similar. This would be based on a defined threshold. Thus, since this is in the denominator for all observations, it is essentially a scaling factor. Thus, when using the results as relative values, this can be ignored in the computations. In this case, we could use the percent of "1" values, or $P(B)=0.02$.

From the binary logit, we compute the probabilities of have 1 or more crashes using the data. For week 1, we get:

$$P(Y>0)=\frac{exp(-3.5)}{1+exp(-3.5)}=0.0293$$

Note that this is larger than average ($0.0293>0.02$). Thus, this observation is MORE likely to have 1 or more crashes due to randomness than average, indicating that $P(A)=P(High, but \ not \ randomly)$ is smaller than the average observation. Thus, the observations with smaller predicted probabilities will have higher probabilities of being high, but not due to randomness when they have crash counts of 1 or more. To caprture this, we can estimate the probabilities using the standard error of the logit and, assuming the differences can be approximated using a normal distribution, we have:

$$z=\frac{P(Y>0)-P(y_i>0|\mu)}{\sigma[P(y_i>0|\mu)]}$$

Where $\sigma[P(y_i>0|\mu)]=\sqrt{P(y_i>0|\mu)(1-P(y_i>0|\mu))}$

So, we get:

$$z=\frac{P(Y>0)-P(y_i>0|\mu)}{\sqrt{P(y_i>0|\mu)(1-P(y_i>0|\mu))}}$$

Assuming $z$ is approximately normally distributed, we can use the CDF for the normal distribution to estimate:

$P(A)=P(High, but \ not \ randomly)=\Phi(z)$

\
If we are only computing this once, we could directly use $P(A)$ to rank and identify locations that are likely to benefit from interventions. However, we have weekly data that can be leveraged to improve the estimates. To do this, we will use Bayes Theorem.

\
Now, we have:
$$P(A)=\Phi(z)=\Phi(\frac{P(Y>0)-P(y_i>0|\mu)}{\sqrt{P(y_i>0|\mu)(1-P(y_i>0|\mu))}})$$

$$P(B|A)=P(High|High, but \ not \ randomly)$$

\

If the outcome is $y_i>0$, then we have a "high" outcome and $P(B_i)=P(A_i)$ for week $i$. If the outcome is $y_i=0$, this indicates that the *reported* number of crashes is 0 which could be different from the true number of crashes. In this case, when $y_i=0$, $P(B_i)=P(A_i)P(\mu_i)$.

\

When we have multiple weeks, we can compute the probability that the road segment had high crash counts over $n$ weeks, all not due to randomness as:

$$P(High, not \ due \ to \ randomness)=P(B)=\Pi_i^n P(B_i)$$

\

Since $P(B_i)$ is dependent on the reported crashes, locations with 1 or more reported crashes over multiple time periods will have the highest values for $P(B)$. Thus, this method accounts for locations that consistently have 1 or more crash counts.

\

For the given data, we have:

$$P(Y_1>0)=\frac{exp(-3.5)}{1+exp(-3.5)}=0.0293$$
$$P(Y_2>0)=\frac{exp(-3.29)}{1+exp(-3.29)}=0.0359$$

$$P(A_1)=\Phi(\frac{P(Y>0)-P(y_i>0|\mu)}{\sqrt{P(y_i>0|\mu)(1-P(y_i>0|\mu))}}))$$
$$=\Phi(\frac{0.02-0.029}{\sqrt{0.029(1-0.029)}})=\Phi(-0.0551)=0.478$$

\

Since $y_1=1$, $P(B_1)=0.478$

$$P(A_2)=\Phi(\frac{P(Y>0)-P(y_i>0|\mu)}{\sqrt{P(y_i>0|\mu)(1-P(y_i>0|\mu))}}))$$
$$=\Phi(\frac{0.02-0.0329}{\sqrt{0.0329(1-0.0329)}})=\Phi(-0.0500)=0.480$$

\

Since $y_1=1$, $P(B_1)=0.466$

\

Using both weeks, we get:

$$P(B_{overall})=P(B_1)P(B_2)=0.229$$

In [43]:
from scipy.stats import norm
mu1 = - 3.5
mu2 = -3.29

pbase = 0.02
p1 = logit_prob(mu1) # probability week 1
p2 = logit_prob(mu2) # probability week 2

z1 = (pbase-p1)/np.sqrt(p1*(1-p1))
z2 = (pbase-p1)/np.sqrt(p2*(1-p2))

In [47]:
pb1 = norm.cdf(z1) # since y = 1, P(B)=P(A)
pb2 = norm.cdf(z2) # since y = 1, P(B)=P(A)

x =np.asarray([pb1, pb2])

prob_B = pb_overall(x)

print("P(B) for week 1 is {}".format(pb1))
print("P(B) for week 2 is {}".format(pb2))
print("P(B) overall is {}".format(prob_B))

P(B) for week 1 is 0.47798701071930316
P(B) for week 2 is 0.48004360794198525
P(B) overall is 0.22945460917509866


Note: We will need to determine how many weeks we want to use and do a rolling timeframe for these calculations